In [25]:
import os
from time import time
import pandas as pd
import tensorflow as tf
from milp import codify_network, codify_network_relaxed
from teste import get_explanation_relaxed, get_minimal_explanation
from typing import List
from docplex.mp.constr import LinearConstraint

In [26]:
def gerar_rede(dir_path: str, num_classes: int, n_neurons: int, n_hidden_layers: int):
    data_train = pd.read_csv(dir_path + "\\" + "train.csv").to_numpy()
    data_test = pd.read_csv(dir_path + "\\" + "test.csv").to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons.h5"
    )

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    print(f"Tempo de Treinamento: {time()-start}")

    # salvar modelo
    model = tf.keras.models.load_model(model_path)

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    model.evaluate(x_test, y_test_ohe, verbose=2)



In [27]:
def explain_instance(
    dataset: {}, configuration: {}, instance_index: int
) -> List[LinearConstraint]:
    dir_path, n_classes, model = (
        dataset["dir_path"],
        dataset["n_classes"],
        dataset["model"],
    )

    method = configuration["method"]
    relaxe_constraints = configuration["relaxe_constraints"]

    data_test = pd.read_csv(f"{dir_path}/test.csv")
    data_train = pd.read_csv(f"{dir_path}/train.csv")

    data = data_train._append(data_test)

    model = tf.keras.models.load_model(f"{dir_path}/{model}")

    # todo: modificar
    # mdl, output_bounds = codify_network(model, data, method, relaxe_constraints)
    mdl_milp_with_binary_variable, output_bounds_binary_variables, bounds_binary_variables = codify_network(
        model, data, method, relaxe_constraints
    )

    # usar bounds precisos do primeiro modelo
    model_milp_relaxed, output_bounds_relaxed = codify_network_relaxed(
        model, data, method, relaxe_constraints, output_bounds_binary_variables, bounds = bounds_binary_variables
    )

    network_input = data.iloc[instance_index, :-1]
    print(network_input)  # network_input = instance

    network_input = tf.reshape(tf.constant(network_input), (1, -1))

    network_output = model.predict(tf.constant(network_input))[0]

    network_output = tf.argmax(network_output)

    mdl_aux = model_milp_relaxed.clone() # todo: testar depois com mdl_milp_with_binary_variable 

    explanation = get_explanation_relaxed(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,
        delta = 0
    )
    return explanation


In [28]:
def gerar_rede_com_dataset_iris(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\iris"
    num_classes = 3
    gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


def gerar_rede_com_dataset_digits(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\digits"
    num_classes = 10
    gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


In [29]:
def explicar_rede():
    datasets = [
        {
            "dir_path": "datasets/digits",
            "model": "models/model_5layers_20neurons.h5",
            "n_classes": 10,
        },
        {
            "dir_path": "datasets/iris",
            "model": "models/model_1layers_20neurons.h5",
            "n_classes": 3,
        },
        {
            "dir_path": "datasets/iris",
            "model": "models/model_6layers_20neurons.h5",
            "n_classes": 3,
        },
    ]
    configurations = [{"method": "fischetti", "relaxe_constraints": True}]

    for i in range(0, 1):
        explanation = explain_instance(
            dataset=datasets[1], configuration=configurations[0], instance_index=i
        )

    for x in explanation:
        print(x)
    print("len: ", len(explanation))

In [30]:
explicar_rede()

[-0.6122007369995119, 1.6778421103954317]
[-1.0509401857852936, 1.2092040181159973]
[-0.868502140045166, 0.32010936737060547]
[-0.21047664433717728, 2.1316187977790833]
[-0.27645808085799217, 1.6028032936155796]
[-0.27311925403773785, 0.8610581457614898]
[-0.5554660968482494, 0.21914495155215263]
[-0.4563867449760437, 0.7460575252771378]
[-0.8218722145767211, 0.07872235774993896]
[-0.851436048746109, 1.2469268441200256]
[-0.5933456420898438, 0.9865902364253999]
[-0.3293287009000778, 1.5459568053483963]
[-0.2331935614347458, 1.6790897846221924]
[-0.5084999693198043, 0.22432684525847438]
[-0.2245442429557443, 1.138442151248455]
[-0.3766915798187256, 1.0976879671216009]
[-0.9108107537031174, 0.41926206648349756]
[-0.5208602622151375, 0.3270818774666865]
[-0.6625148589096069, 0.0]
[-0.33704200387001043, 1.2212621569633484]
[-0.6122007369995119, 1.6778421103954317]
[-1.0509401857852936, 1.2092040181159973]
[-0.868502140045166, 0.32010936737060547]
sepal_length    0.138889
sepal_width     0.